In [5]:
import os.path as op
import matplotlib.pyplot as plt
import socket
import torch
from torch.utils.data import DataLoader
import logging
from dataset import DataloaderImg, GreyToRGB, NormalizeVolume
from torch import optim
import torchvision.transforms.v2 as tfms
from tqdm import tqdm
from torch.utils import tensorboard
from summary import summary_grid, show_batch
from loss import DiceLoss
from validation import get_validation_loop_len, generalized_dsc, accuracy

plt.ion()  # interactive mode

# install pytorch correctly
# https://discuss.pytorch.org/t/torch-cuda-is-not-available/74845/11

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
logging.basicConfig(
    level=logging.INFO,
)
# install cuda driver ubuntu
# https://ubuntu.com/server/docs/nvidia-drivers-installation

cpu


In [8]:
# https://pytorch.org/docs/stable/cuda.html
# torch.cuda.is_initialized()

if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    print(f"Current cuda device: {torch.cuda.get_device_name(current_device)}")

In [3]:
# script for segmentation
# https://github.com/mateuszbuda/brain-segmentation-pytorch

In [136]:
# help in loading images
# https://discuss.pytorch.org/t/how-to-load-all-the-nii-from-the-directory-without-augmentation-using-pytorch-dataloader/60938/3
root_dir = "/media/MeMoSLAP_Subjects/derivatives/automated_electrode_extraction"  #!NOTE: delete "/train" for all subjects

preprocessing = [NormalizeVolume(), GreyToRGB()]
transforms = [tfms.RandomRotation(180)]
n_validation = 4
# full_dataset = Dataloder_img('C:/Users/Ali ktk/.spyder-py3/dataloader/data/train/1', 'C:/Users/Ali ktk/.spyder-py3/dataloader/data/train/1/ADNI_136_S_0300_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080529142830882_S50401_I107759.nii' ,tfms.Compose([tfms.RandomRotation(180).tfms.ToTensor()]))
train_dataset = DataloaderImg(
    root_dir,
    subset="train",
    validation_cases=n_validation,
    preprocessing=preprocessing,
    transforms=transforms,
)
validation_dataset = DataloaderImg(
    root_dir,
    subset="validation",
    validation_cases=n_validation,
    preprocessing=preprocessing,
    transforms=transforms,
)

loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [ ]:
# train_img, train_mask = next(iter(loader))
# show_batch(train_img, train_mask)
# plt.show()

In [5]:
unet = torch.hub.load(
    "mateuszbuda/brain-segmentation-pytorch",
    "unet",
    in_channels=3,
    out_channels=1,
    init_features=32,
    pretrained=False,
)

Using cache found in /home/riemanns/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


Check free GPU memory with `nvtop` in the terminal


In [33]:
unet.to(device)

batch_size = 16
if batch_size > 16:
    raise ValueError("Batch size too large for 6GB of GPU memory.")
phase = "train"
epochs = 200
img_freq = 10
lr = 2e-3

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

loaders = {"train": train_loader, "valid": valid_loader}

dsc_loss = DiceLoss()
best_validation_dsc = 0.0

optimizer = optim.Adam(unet.parameters(), lr=lr)

loss_train = []

step = 0

writer = tensorboard.SummaryWriter()

for _ in tqdm(range(epochs), total=epochs):
    unet.train()

    for i, data in enumerate(loaders[phase]):
        step += 1

        x, y_true = data
        x, y_true = x.to(device, dtype=torch.float), y_true.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == "train"):
            y_pred = unet(x)
            loss = dsc_loss(y_pred, y_true)

            loss_train.append(loss.item())
            loss.backward()
            optimizer.step()

            writer.add_scalar("loss/train", loss.item(), step)

        if step % img_freq != 0:
            continue

        writer.add_figure(
            f"step {step}: predictions vs. petra",
            summary_grid(x.cpu().detach().type(torch.IntTensor), y_pred.cpu().detach()),
            global_step=step,
        )

writer.flush()

  0%|          | 0/200 [00:00<?, ?it/s]

0.9881510138511658
0.9859606027603149


  0%|          | 0/200 [00:57<?, ?it/s]


KeyboardInterrupt: 

In [107]:
torch.save(unet, f"unet_epochs_{epochs}_batchsize_{batch_size}_{lr}.pt")

In [6]:
unet = torch.load("unet_epochs_1000_batchsize_15.pt")

In [40]:
mask = np.random.choice([0, 1], size=(4, 1, 288, 288), p=[0.9, 0.1])
pred = mask + np.random.choice([0, 1], size=(4, 1, 288, 288), p=[0.9, 0.1])
pred = pred.clip(0, 1)

accuracy(pred, mask)

array([0.90956549, 0.91050588, 0.91048177, 0.90899884])

In [114]:
acc_matrix = np.zeros((len(validation_dataset), validation_dataset.n_slices))
dsc_matrix = np.zeros((len(validation_dataset), validation_dataset.n_slices))

In [139]:
unet.eval()

bs = 4
valloader = DataLoader(validation_dataset, batch_size=bs, shuffle=False)
loop_len = get_validation_loop_len(validation_dataset, bs)

start_idx = 0
idx_step = bs if bs < len(validation_dataset) else len(validation_dataset)

val_slice = validation_dataset.val_slice

for i in tqdm(range(loop_len)):
    for x, y_true in valloader:
        validation_dataset.val_slice = val_slice

        x, y_true = x.to(device, dtype=torch.float), y_true.to(device)
        y_pred = unet(x)

        y_pred, y_true = y_pred.cpu().detach().numpy(), y_true.cpu().detach().numpy()

        acc_matrix[
            start_idx : start_idx + idx_step, validation_dataset.val_slice
        ] = accuracy(y_pred, y_true)

        dsc_matrix[
            start_idx : start_idx + idx_step, validation_dataset.val_slice
        ] = generalized_dsc(y_pred, y_true)

        start_idx += bs
        if start_idx >= len(validation_dataset):
            start_idx = 0
            idx_step = bs
            val_slice += 1

  0%|          | 0/224 [00:00<?, ?it/s]/tmp/ipykernel_2250977/3737991106.py:2: RuntimeWarning: divide by zero encountered in divide
  weight_el = 1 / np.sum(mask, axis=(2, 3))
/tmp/ipykernel_2250977/3737991106.py:12: RuntimeWarning: invalid value encountered in divide
  return (2 * (pred * mask * weight).sum(axis=(1, 2, 3))) / ((pred + mask) * weight).sum(axis=(1, 2, 3))
 70%|███████   | 157/224 [16:28<07:38,  6.84s/it]

In [132]:
acc_matrix.mean(axis=1)

array([0.99952684, 0.99953029, 0.99946748, 0.9995952 ])

In [138]:
dsc2_matrix.mean(axis=1)

array([0.13217167, 0.1242136 , 0.06574694, 0.07564986])

In [ ]:
inference_with_net = "unet_epochs_1000_batchsize_15.pt"

inference_dataset = DataloaderImg(
    training_dir,
    subset="inference",
    preprocessing = preprocessing 
)
bs = 8
inference_loader = DataLoader(inference_dataset, batch_size=bs, shuffle=False)
loop_len = get_validation_loop_len(inference_dataset, bs)

start_idx = 0
idx_step = bs if bs < len(validation_dataset) else len(validation_dataset)
inf_slice = 0

inferences = np.zeros(
    (
        len(inference_dataset), 
        inference_dataset.n_slices, 
        inference_datset.img_dim[0], 
        inference_dataset.img_dim[1],
    )
)

with torch.grad_enabled(False):
    
    unet = torch.hub.load(
        "mateuszbuda/brain-segmentation-pytorch",
        "unet",
        in_channels=3,
        out_channels=1,
        init_features=32,
        pretrained=False,
    )
    
    unet.load(inference_with_net)
    
    unet.eval()
    unet.to_device()
    

    for i in tqdm(range(loop_len)):
        for x, _ in inference_loader:
            inference_dataset.val_slice = inf_slice
            x = x.to(device, dtype=torch.float)
            y_pred = unet(x)
            
            y_pred = y_pred.cpu().detach().numpy()
            
            inferences[
                start_idx : start_idx + idx_step, inference_dataset.val_slice
            ] = y_pred 

            start_idx += bs
            
            if start_idx >= len(inference_dataset):
                start_idx = 0
                idx_step = bs
                inf_slice += 1
